In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mtl
import autogluon as ag
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from autogluon.tabular import TabularPredictor,TabularDataset
from autogluon.features.generators import AutoMLPipelineFeatureGenerator

C:\ProgramData\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_all = pd.read_csv("Downloads/archive (3)/continuous dataset_tabular.csv")
print("There are %0.0f" %df_all.shape[0] + " repeated measures and %0.0f" %df_all.shape[1] +" variables in the dataset" )
df_all.head()

There are 48048 repeated measures and 17 variables in the dataset


,datetime,nat_demand,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,QV2M_dav,TQL_dav,W2M_dav,Holiday_ID,holiday,school
0,1/3/2015 0:00,960.3360,25.884526,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0
1,1/3/2015 1:00,970.3450,25.865259,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0
2,1/3/2015 2:00,912.1755,25.899255,0.018653,0.016418,22.166944,23.399255,0.017265,0.001327,10.681517,22.578943,0.016509,0.087646,5.572471,0,0,0
3,1/3/2015 3:00,900.2688,25.937280,0.018768,0.015480,22.454911,23.343530,0.017211,0.001428,10.874924,22.531030,0.016479,0.078735,5.871184,0,0,0
4,1/3/2015 4:00,889.9538,25.957544,0.018890,0.016273,22.110481,23.238794,0.017128,0.002599,10.518620,22.512231,0.016487,0.068390,5.883621,0,0,0


In [ ]:
adf = adfuller(df_train)
print('adfuller test P-Value: ', adf[1])
#df_all.reset_index(inplace=True)

In [71]:
if p_value <= 0.05:
    print("Reject the null hypothesis: The time series is likely stationary.")
else:
    print("Fail to reject the null hypothesis: The time series is likely non-stationary.")

Reject the null hypothesis: The time series is likely stationary.


In [3]:
df_all['datetime'] = pd.to_datetime(df_all['datetime'],format="%m/%d/%Y %H:%M")

In [4]:
correlation_matrix = df_all.corr()
correlation_ = correlation_matrix['nat_demand'].sort_values(ascending=False)

In [5]:
print(correlation_)

nat_demand    1.000000
T2M_toc       0.652818
T2M_dav       0.648283
T2M_san       0.627031
W2M_san       0.191757
datetime      0.153810
TQL_san       0.119063
W2M_toc       0.098398
TQL_toc       0.073126
TQL_dav       0.042054
school        0.040094
QV2M_san      0.022193
QV2M_dav     -0.002095
QV2M_toc     -0.036686
W2M_dav      -0.054830
Holiday_ID   -0.129832
holiday      -0.165671
Name: nat_demand, dtype: float64


In [6]:
df_all=df_all.drop(['QV2M_dav','QV2M_toc','W2M_dav','TQL_dav','QV2M_san','W2M_toc','TQL_toc','school','Holiday_ID'],axis=1)

In [7]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48048 entries, 0 to 48047
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    48048 non-null  datetime64[ns]
 1   nat_demand  48048 non-null  float64       
 2   T2M_toc     48048 non-null  float64       
 3   T2M_san     48048 non-null  float64       
 4   TQL_san     48048 non-null  float64       
 5   W2M_san     48048 non-null  float64       
 6   T2M_dav     48048 non-null  float64       
 7   holiday     48048 non-null  int64         
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 2.9 MB


In [8]:
df_all.head()

,datetime,nat_demand,T2M_toc,T2M_san,TQL_san,W2M_san,T2M_dav,holiday
0,2015-01-03 00:00:00,960.3360,25.884526,23.482446,0.001855,10.328949,22.662134,0
1,2015-01-03 01:00:00,970.3450,25.865259,23.482446,0.001855,10.328949,22.662134,0
2,2015-01-03 02:00:00,912.1755,25.899255,23.399255,0.001327,10.681517,22.578943,0
3,2015-01-03 03:00:00,900.2688,25.937280,23.343530,0.001428,10.874924,22.531030,0
4,2015-01-03 04:00:00,889.9538,25.957544,23.238794,0.002599,10.518620,22.512231,0


In [9]:
train_data = df_all[df_all['datetime'] < '2020-01-01']
test_data = df_all[df_all['datetime'] >= '2020-01-01']
print(train_data.shape,test_data.shape)

(43776, 8) (4272, 8)


In [10]:
train_data=train_data.set_index('datetime')
test_data=test_data.set_index('datetime')

In [11]:
train_data.head(2)

,nat_demand,T2M_toc,T2M_san,TQL_san,W2M_san,T2M_dav,holiday
datetime,,,,,,,
2015-01-03 00:00:00,960.336,25.884526,23.482446,0.001855,10.328949,22.662134,0
2015-01-03 01:00:00,970.345,25.865259,23.482446,0.001855,10.328949,22.662134,0


In [12]:
test_data.tail(2)

,nat_demand,T2M_toc,T2M_san,TQL_san,W2M_san,T2M_dav,holiday
datetime,,,,,,,
2020-06-26 22:00:00,1081.568,26.971155,25.393030,0.144531,2.396369,23.658655,0
2020-06-26 23:00:00,1041.624,26.867487,25.258112,0.108063,2.720871,23.601862,0


In [13]:
train_data=TabularDataset(train_data)
test_data=TabularDataset(test_data)

In [14]:
predictor=TabularPredictor(label='nat_demand',problem_type='regression')
predictor.fit(train_data,hyperparameters="default")

No path specified. Models will be saved in: "AutogluonModels\ag-20231219_023922"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231219_023922"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.9.18
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          8
Memory Avail:       

[1000]	valid_set's rmse: 120.171
[2000]	valid_set's rmse: 119.573
[3000]	valid_set's rmse: 119.234
[4000]	valid_set's rmse: 119.118
[5000]	valid_set's rmse: 119.193


	-119.0865	 = Validation score   (-root_mean_squared_error)
	16.97s	 = Training   runtime
	0.59s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 120.362


	-120.0767	 = Validation score   (-root_mean_squared_error)
	2.88s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-119.3909	 = Validation score   (-root_mean_squared_error)
	70.74s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: CatBoost ...
	-119.1197	 = Validation score   (-root_mean_squared_error)
	49.38s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-118.3037	 = Validation score   (-root_mean_squared_error)
	10.86s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-120.7141	 = Validation score   (-root_mean_squared_error)
	104.75s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: XGBoost ...
	-120.1533	 = Validation score   (-root_mean_squared_error)
	2.68s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-120.6355	 = Validation score   (-root_mean_squared_error)
	293.91s	 = Training   runtime
	0.0

In [44]:
#test_data1=test_data.drop('nat_demand',axis=1)

In [15]:
predictions=predictor.predict(test_data)

In [16]:
result=pd.DataFrame({'Actual':test_data['nat_demand'],'Predicted':predictions})

In [17]:
print(result)

                        Actual    Predicted
datetime                                   
2020-01-01 00:00:00  1032.2630   988.878784
2020-01-01 01:00:00  1017.5806  1002.342712
2020-01-01 02:00:00  1000.2797   985.992432
2020-01-01 03:00:00   987.2383   973.020813
2020-01-01 04:00:00   970.3667   968.823730
...                        ...          ...
2020-06-26 19:00:00  1127.2491  1200.955566
2020-06-26 20:00:00  1128.5592  1113.822144
2020-06-26 21:00:00  1112.7488  1093.193481
2020-06-26 22:00:00  1081.5680  1077.067749
2020-06-26 23:00:00  1041.6240  1070.476074

[4272 rows x 2 columns]


In [18]:
predictor.evaluate(test_data, silent=True)

{'root_mean_squared_error': -170.80496143770918,
 'mean_squared_error': -29174.33485173732,
 'mean_absolute_error': -137.08858039180456,
 'r2': 0.059284961658329194,
 'pearsonr': 0.48179486554549267,
 'median_absolute_error': -117.24463828124988}

In [20]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetFastAI,-170.036774,-120.714097,root_mean_squared_error,0.156650,0.125428,104.749911,0.156650,0.125428,104.749911,1,True,8
1,ExtraTreesMSE,-170.229304,-118.303724,root_mean_squared_error,0.377346,0.125796,10.856193,0.377346,0.125796,10.856193,1,True,7
2,LightGBMLarge,-170.236274,-119.574003,root_mean_squared_error,0.070539,0.029655,2.779881,0.070539,0.029655,2.779881,1,True,11
3,CatBoost,-170.629437,-119.119700,root_mean_squared_error,0.031247,0.022552,49.376525,0.031247,0.022552,49.376525,1,True,6
4,RandomForestMSE,-170.740481,-119.390898,root_mean_squared_error,0.388424,0.133334,70.740269,0.388424,0.133334,70.740269,1,True,5
5,WeightedEnsemble_L2,-170.804961,-117.686685,root_mean_squared_error,1.449212,0.968809,442.692914,0.010013,0.003173,0.691234,2,True,12
6,XGBoost,-171.104590,-120.153302,root_mean_squared_error,0.078550,0.015641,2.679356,0.078550,0.015641,2.679356,1,True,9
7,LightGBM,-171.299964,-120.076686,root_mean_squared_error,0.078545,0.046890,2.882931,0.078545,0.046890,2.882931,1,True,4
8,LightGBMXT,-172.335974,-119.086497,root_mean_squared_error,0.573347,0.592755,16.972478,0.573347,0.592755,16.972478,1,True,3
9,NeuralNetTorch,-176.517169,-120.635531,root_mean_squared_error,0.028828,0.031653,293.914693,0.028828,0.031653,293.914693,1,True,10
